# Implementation of "Rational Criteria for Diagonalizability of Real Matrices"
## Carlos A.C.C. Perello (Imperial College London)
## Original work by João Ferreira Alves (ULisboa IST)

In [2]:
using LinearAlgebra

MethodError: MethodError: no method matching ^(::Matrix{Float64}, ::UnitRange{Int64})
Closest candidates are:
  ^(::AbstractMatrix, !Matched::Integer) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/dense.jl:445
  ^(::AbstractMatrix{T}, !Matched::Real) where T at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/dense.jl:488
  ^(::AbstractMatrix, !Matched::Number) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/dense.jl:529

In [89]:
function frob_ip(A,B)
    ret = tr(A*B')
    ret
end

frob_ip (generic function with 1 method)

In [90]:
function Kₚ(A, p)
    mat = zeros(p,p)
    row = [A^i for i = 0:p-1]
    for i = 1:p
        mat[i,:] = tr.(row)
        row = [A*r for r in row] 
    end
    mat
end

function Lₚ(A, p)
    mat = zeros(p, p)
    row = [A'^i for i = 0:p-1]
    for i = 1:p
        mat[i,:] = tr.(row)
        row = [A*r for r in row]
    end
    mat
end

Lₚ (generic function with 1 method)

In [131]:
function r(A)
    for p=1:size(A)[1]
        if det(Kₚ(A, p)) ≤ 0
            return p
        end
    end
end

r(Diagonal(1:12))


10